In [1]:
import numpy as np
import pandas as pd
import matplotlib

import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({"font.size":16})
import os
import sys

sys.path.append(os.path.abspath('../..'))
from modules import io
import modules.vascular_data as sv
from scipy.interpolate import Rbf
import vtk

In [2]:
CSVS = [
    "/media/marsdenlab/Data1/UQ/0110/csv/steady/aorta_tube.csv",
    "/media/marsdenlab/Data1/UQ/0110/csv/steady/right_iliac_tube.csv",
]

FIG_DIR = "/media/marsdenlab/Data1/UQ/0110/figures/3dplots"

BASE_VTU = "/media/marsdenlab/Data1/UQ/0110/generations/\
1/coarse/0/exterior.vtp"

generations = [1,2,3,4]
meshes  = ["coarse"]
quants = [
    "area",
    "radius_actual",
    "pressure_0",
    "vWSS_3_boundary",
    "velocity_3"
]

dfs = []
for c in CSVS:
    d = pd.read_csv(c)
    dfs.append(d)
    
df = pd.concat(dfs)

In [3]:
for g in generations:
    for m in meshes:
        print(g,m)
        vtu = sv.vtk_read_native_polydata(BASE_VTU)
        
        data = df.loc[ (df['generation']==g) & (df['mesh']==m)]

        grouped = data.groupby(["path","point"])

        mu  = grouped.mean()
        std = grouped.std()

        x = mu['x'].values
        y = mu['y'].values
        z = mu['z'].values
        
        for q in quants:
            print(q)
            v1 = std[q]
            v2 = std[q].values/mu[q].values

            interp1 = Rbf(x,y,z,v1, function='linear')
            interp2 = Rbf(x,y,z,v2, function='linear')
            
            for interp,label in zip([interp1,interp2], ['_std','_std_relative']):
                array = vtk.vtkDoubleArray()
                array.SetNumberOfComponents(1)
                array.SetNumberOfTuples(vtu.GetNumberOfPoints())
                for i in range(vtu.GetNumberOfPoints()):
                    point = vtu.GetPoint(i)
                    v = interp(*point)
                    array.SetValue(i, v)

                vtu.GetPointData().AddArray(array)
                array.SetName(q+label)
        
        sv.vtk_write_native_polydata(vtu,FIG_DIR+'/{}_{}.vtp'.format(g,m))

1 coarse
area
radius_actual
pressure_0
vWSS_3_boundary
velocity_3
2 coarse
area
radius_actual
pressure_0
vWSS_3_boundary
velocity_3
3 coarse
area
radius_actual
pressure_0
vWSS_3_boundary
velocity_3
4 coarse
area
radius_actual
pressure_0
vWSS_3_boundary
velocity_3
